In [ ]:
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from create_model import create_model

In [3]:
model_path = 'gs://iam-model-staging/run_55_train/model/'
pred_model = keras.models.load_model(model_path)

In [4]:
BATCH_SIZE=128
KERNEL_SIZE=4
ACTIVATION_FUNCTION='relu'
LEARNING_RATE=0.001
NUM_UNITS_DENSE=256
NUM_UNITS_LTSM2=1024
### run 55 parameters
DROPOUT=0.12489316869910207
NUM_UNITS_LTSM1=512
### run 41 parameters
# DROPOUT=0.1
# NUM_UNITS_LTSM1=768
pred_model.save_weights('pred')
full_model = create_model(KERNEL_SIZE, ACTIVATION_FUNCTION, NUM_UNITS_DENSE, DROPOUT, NUM_UNITS_LTSM1, NUM_UNITS_LTSM2, LEARNING_RATE)
full_model.g

Model: "ocr_model_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 400, 100, 1  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 400, 100, 32  544         ['image[0][0]']                  
                                )                                                                 
                                                                                                  
 pool1 (MaxPooling2D)           (None, 200, 50, 32)  0           ['Conv1[0][0]']                  
                                                                                       

In [ ]:
pred_model.trainable = False
img_input = layers.Input(shape=(400, 100, 1), name='image', dtype='float32')  # TODO: don't hardcode this
labels = layers.Input(name='label', shape=(None,), dtype='float32')
full_model_output = CTCLayer(name='ctc_loss')(labels,img_input)
full_model = keras.models.Model(inputs=[pred_model.input, labels], outputs=full_model_output, name='model_transfer_learning')
full_model.compile(keras.optimizers.Adam(learning_rate=learning_rate))

<class 'keras.engine.keras_tensor.KerasTensor'>


ValueError: Exception encountered when calling layer "ctc_loss" (type CTCLayer).

in user code:

    File "/tmp/ipykernel_1/3066800124.py", line 16, in call  *
        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
    File "/opt/conda/lib/python3.7/site-packages/keras/backend.py", line 6580, in ctc_batch_cost
        y_pred = tf.math.log(tf.compat.v1.transpose(y_pred, perm=[1, 0, 2]) + epsilon())

    ValueError: Dimension must be 4 but is 3 for '{{node ctc_loss/transpose_2}} = Transpose[T=DT_FLOAT, Tperm=DT_INT32](Placeholder_1, ctc_loss/transpose_2/perm)' with input shapes: [?,400,100,1], [3].


Call arguments received:
  • y_true=tf.Tensor(shape=(None, None), dtype=float32)
  • y_pred=tf.Tensor(shape=(None, 400, 100, 1), dtype=float32)

In [ ]:
pred_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 400, 100, 1)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 400, 100, 32)      544       
                                                                 
 pool1 (MaxPooling2D)        (None, 200, 50, 32)       0         
                                                                 
 Conv2 (Conv2D)              (None, 200, 50, 64)       32832     
                                                                 
 pool2 (MaxPooling2D)        (None, 100, 25, 64)       0         
                                                                 
 reshape (Reshape)           (None, 100, 1600)         0         
                                                                 
 dense1 (Dense)              (None, 100, 256)          409856